# Sistema de Informação de Agravos de Notificação (SINAN)

Esse notebook tem como objetivo realizar o ELT dos dados provenientes do SINAN que serão utilizados no projeto Dengue.

### Libs

In [1]:
%run ../config/bootstrap.py

In [2]:
import pandas as pd
from utils import get_project_root, save_parquet_in_chunks , load_partitioned_parquet
import basedosdados as bd

In [3]:
project_root = get_project_root() 
#billing_id = 'ufrgs-ppgc-dengue'

In [4]:
#%load_ext sql
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaylimit = 100
%sql bigquery://ufrgs-ppgc-dengue

Connecting to 'bigquery://ufrgs-ppgc-dengue'

# Datasets

In [13]:
%%sql sinan <<
SELECT ano
,id_estabelecimento as id_cnes
,  sum(CASE WHEN LOWER(classificacao_final) LIKE '%dengue%' THEN 1 ELSE 0 END) AS target
,  sum(CASE WHEN LOWER(evolucao_caso) LIKE '%óbito%' THEN 1 ELSE 0 END) AS obito
,  sum(CASE WHEN data_encerramento is not null THEN 1  ELSE 0  END) AS flag_caso_encerrado
,count(*) as qt

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano>2020
group by 1,2


Running query in 'bigquery://ufrgs-ppgc-dengue'

149258 rows affected.

In [10]:
sinan.head()

,ano,id_cnes,target,obito,flag_caso_encerrado,qt
0,2024,0902179,0,0,1,252
1,2024,2000083,0,0,1,144
2,2024,2000334,0,0,1,57
3,2024,2000636,0,0,1,352
4,2024,2006960,0,0,1,66


In [11]:
path = project_root / "data/4_gold/dim_cnes/dim_cnes_part_1.parquet"
cnes = pd.read_parquet(path)
cnes.head()

,id_cnes,id_unidade_cnes,nu_cnpj_mantenedora,id_pfpj,nivel_dep,no_razao_social,no_fantasia,no_logradouro,nu_endereco,no_complemento,no_bairro,id_cep,id_regiao_saude,id_micro_regiao,id_distrito_sanitario,id_distrito_administrativo,nu_telefone,nu_fax,no_email,nu_cpf,nu_cnpj,id_atividade,id_clientela,nu_alvara,data_expedicao,id_orgao_expedidor,data_val_lic_sani,id_lic_sani,id_unidade,id_turno_atendimento,id_estado_gestor,id_municipio_gestor,data_atualizacao,id_usuario,id_cpfdiretorcln,reg_diretorcln,st_adesao_filantrop,id_motivo_desab,no_url,nu_latitude,nu_longitude,data_atualizacao_geolocalizacao,no_usuario_geo,id_natureza_jur,id_estab_sempre_aberto,st_geracredito_gerente_sgif,st_conexao_internet,id_tipo_unidade,no_fantasia_abrev,id_gestao,data_atualizacao_origem,id_tipo_estabelecimento,id_atividade_principal,st_contrato_formalizado,id_tipo_abrangencia,st_coworking,hash,data_inicio_vigencia_silver,data_fim_vigencia_silver,data_atualizacao_silver,vigencia_silver
0,4013484,2408104013484,08241754000145,3,3,RIO GRANDE DO NORTE SECRETARIA DA SAUDE PUBLICA,HOSPITAL GISELDA TRIGUEIRO,CONEGO MONTE,110,None,QUINTAS,59037170,7,None,04,None,32327915,None,None,None,08241754010965,03,03,None,None,None,None,None,05,06,24,240810,2025-08-07 00:00:00,SAUDE,68051085353,4363,None,None,None,"-5,794974298299524","-35,209068059921265",2021-12-06 00:00:00,4013484,1023,S,None,S,None,None,D,2003-10-30 00:00:00,006,009,S,None,N,4c98b01f2fd50a2da5a7fcb49580d51674ee156797054d93dc8b3b301afade8b,2025-08-17,9999-12-31,2025-08-17,1
1,2589648,5201402589648,37942539000170,3,3,FUNDO MUNICIPAL DE SAUDE DE APDA GO,UPA BRASICON,RUA DAS ACASIA,S/N,ESQ C AV CHILE,RESIDENCIAL BRASICON,74975390,001,None,None,None,3545 9953,None,None,None,None,04,03,None,None,None,None,None,73,06,52,520140,2024-07-26 00:00:00,YASMIN,03258865124,None,None,None,None,"-16,82760","-49,24209",2024-07-26 00:00:00,YASMIN,1031,S,None,S,None,None,M,2002-05-22 00:00:00,008,007,None,None,None,f70e972e8d235757c25415e536078cb9b4e8e1b93b4d07280c2e961b8154df77,2025-08-17,9999-12-31,2025-08-17,1
2,2052474,3549802052474,None,3,1,UNIMED SJRPRETO COOPERATIVA DE TRABALHO MEDICO,COMPLEXO DE SAUDE UNIMED SAO JOSE DO RIO PRETO,AVENIDA BADY BASSITT,4870,None,JAD ALTO RIO PRETO,15025000,0215,None,None,None,17 3202-1110,None,unimed@unimedripreto.com.br,None,45100138000362,04,03,354980501-861-001673-1-2,2024-09-13 00:00:00,2,2025-09-13 00:00:00,1,73,06,35,354980,2025-06-04 00:00:00,FERNANDA,25123834826,None,None,None,None,"-20,819074142767942","-49,397743074317205",2025-04-01 00:00:00,FERNANDA,2143,S,None,S,None,None,M,2003-02-25 00:00:00,008,007,N,None,None,66339a46703b49c1bab3c25113d169321010c222f882e1944b11b9d5363cae9d,2025-08-17,9999-12-31,2025-08-17,1
3,6270093,3549806270093,46588950000180,3,3,PREFEITURA MUNICIPAL DE SAO JOSE DO RIO PRETO,UPA REGIAO NORTE,R MANOEL MORENO ESQ R JOSEPHA VOLTARELLI SANFELICE,130,None,JARDIM ANTUNES,15047050,0215,None,None,None,17-32375315,None,None,None,None,03,03,354980501-861-002357-1-7,2018-02-23 00:00:00,2,2019-02-23 00:00:00,1,73,06,35,354980,2025-08-08 00:00:00,FERNANDA,21728064805,473613,None,None,None,"-20,777456","-49,403220",2019-10-01 00:00:00,FERNANDA,1244,S,None,S,None,None,M,2009-06-23 00:00:00,008,007,None,None,None,245706442b1794984d4002593ef3ea47d4e082ca8ca5a07ffbe07421cdc06207,2025-08-17,9999-12-31,2025-08-17,1
4,2589613,5201402589613,37942539000170,3,3,FUNDO MUNICIPAL DE SAUDE DE APARECIDA DE GOIANIA,CAIS NOVA ERA,AVENIDA SAO JOAO,S/N,None,JARDIM NOVA ERA,74916970,001,None,None,None,(62)3545 5925,None,None,None,None,04,03,None,2024-01-31 00:00:00,2,2024-12-31 00:00:00,1,20,06,52,520140,2025-01-28 00:00:00,YASMIN,01162515147,None,None,None,None,"-16,7474832","-49,2812776",2024-02-01 00:00:00,MARCIA,1031,S,None,S,None,None,M,2002-05-22 00:00:00,008,007,S,None,None,603e683c06ba7c1292ed6742e66b4bbc3e50da43cafb25e931b6f944e1324839,2025-08-17,9999-12-31,2025-08-17,1


In [14]:
sinan_cnes = sinan.merge(cnes, on="id_cnes", how="left")
sinan_cnes = sinan_cnes.drop_duplicates()

In [15]:
sinan_cnes.head()

,ano,id_cnes,target,obito,flag_caso_encerrado,qt,id_unidade_cnes,nu_cnpj_mantenedora,id_pfpj,nivel_dep,no_razao_social,no_fantasia,no_logradouro,nu_endereco,no_complemento,no_bairro,id_cep,id_regiao_saude,id_micro_regiao,id_distrito_sanitario,id_distrito_administrativo,nu_telefone,nu_fax,no_email,nu_cpf,nu_cnpj,id_atividade,id_clientela,nu_alvara,data_expedicao,id_orgao_expedidor,data_val_lic_sani,id_lic_sani,id_unidade,id_turno_atendimento,id_estado_gestor,id_municipio_gestor,data_atualizacao,id_usuario,id_cpfdiretorcln,reg_diretorcln,st_adesao_filantrop,id_motivo_desab,no_url,nu_latitude,nu_longitude,data_atualizacao_geolocalizacao,no_usuario_geo,id_natureza_jur,id_estab_sempre_aberto,st_geracredito_gerente_sgif,st_conexao_internet,id_tipo_unidade,no_fantasia_abrev,id_gestao,data_atualizacao_origem,id_tipo_estabelecimento,id_atividade_principal,st_contrato_formalizado,id_tipo_abrangencia,st_coworking,hash,data_inicio_vigencia_silver,data_fim_vigencia_silver,data_atualizacao_silver,vigencia_silver
0,2024,2103311,0,0,338,338,3159502103311,18413187000110,3,3,PREFEITURA MUNICIPAL DE SANTA RITA DO ITUETO,CENTRO DE SAUDE DE SANTA RITA DO ITUETO,RUA MARIA INES,175,None,CENTRO,35225000,0016,None,None,None,(33)32651184,None,secretaria.sri@gmail.com,None,None,04,01,None,None,None,None,None,02,04,31,315950,2025-04-15 00:00:00,SAUDE,04850919685,159687,None,None,None,"-19,35961","-41,37983",2023-11-08 00:00:00,APARECIDA,1244,N,None,S,None,None,M,2001-10-30 00:00:00,001,012,None,03,N,7fef0595a0b56a04a9b69d64a28be60b5143d850bf11187833b131334c0bc0eb,2025-08-17,9999-12-31,2025-08-17,1.0
1,2024,2119420,0,0,2562,2562,3108602119420,18017442000106,3,3,MUNICIPIO DE BRASILIA DE MINAS,HOSPITAL MUNICIPAL SENHORA SANTANA,AVENIDA BIAS FORTES,S/N,CENTRO,DONA JOAQUINA,39330000,0030,None,None,None,(38)32311803,None,hmss@brasiliademinas.mg.gov.br,None,None,04,03,0081/03,2003-07-17 00:00:00,None,None,None,05,06,31,310860,2025-05-08 00:00:00,SMSBM,45734399604,None,None,None,None,"-16,219413481866532","-44,42647933959961",2019-05-08 00:00:00,SMSBM,1244,S,None,S,None,None,M,2002-03-12 00:00:00,006,009,None,05,N,c8c883400a30f53a019ecae520aa979455ff55e4afb3e0bfdd2159869d28688c,2025-08-17,9999-12-31,2025-08-17,1.0
2,2024,2144654,0,0,384,384,3122602144654,None,3,1,IRMANDADE NOSSA SENHORA DAS GRACAS DOM JOAQUIM,HOSPITAL NOSSA SENHORA DAS GRACAS DOM JOAQUIM,RUA DR ARY ASCENCAO D OLIVEIRA,85,None,CENTRO,35865000,0009,None,None,None,(31)38661131,None,None,None,22056741000127,04,01,01,2020-12-08 00:00:00,2,2020-12-31 00:00:00,1,05,06,31,312260,2025-04-29 00:00:00,SAUDE,01192593626,74739,2,None,None,"-18,959823","-43,251191",2019-07-12 00:00:00,ELIANE,3999,S,None,S,None,None,M,2002-03-21 00:00:00,006,009,S,03,N,312f8c3c6a5ed01cca4e3ff8a7f2e0f8c90993a493efc99a6e6af9f2af8d9fbf,2025-08-17,9999-12-31,2025-08-17,1.0
3,2024,2307898,0,0,76,76,2102302307898,06052138000110,3,3,PREFEITURA MUNICPAL DE BURITI BRAVO,HOSPITAL MUNICIPAL ZUZA COELHO,RUA JOAQUIM LEITE,S/N,None,LARANJEIRAS,65685000,13,None,None,None,(09)85721010,None,None,None,None,04,01,None,None,None,None,None,05,06,21,210230,2025-06-26 00:00:00,CNESBB,03764609389,None,None,None,None,"-5,829987","-43,834312",2019-09-02 00:00:00,CNESBB,1244,S,None,S,None,None,M,2001-10-30 00:00:00,006,001,None,None,N,8105bef67dee2f643327b0bc728d4a8bb85d46fc31842eca466bb0031ca05dd1,2025-08-17,9999-12-31,2025-08-17,1.0
4,2024,2461625,0,0,25,25,2111742461625,01566688000134,3,3,PREFEITURA MUNICIPAL DE ALEXANDRE COSTA,HOSPITAL MUNICIPAL MARIA JOSE MACHADO,RUA CONEGO ADERSON,S/N,None,CENTRO,65783000,14,None,None,None,(99) 5691073,None,None,None,None,04,01,None,None,None,None,None,05,06,21,211174,2022-08-15 00:00:00,MESTRE,10458875368,None,None,10,None,"-5,258764391608838","-44,054274559020996",2020-02-20 00:00:00,MESTRE,1244,S,None,S,None,None,M,2003-11-09 00:00:00,006,009,None,None,None,e4d79c86fb2fdee65f018b1e7c3b7f3b126535b19279e4c2ded0a50b4d020911,2025-08-17,9999-12-31,2025-08-17,1.0


In [17]:
sinan_cnes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149258 entries, 0 to 149257
Data columns (total 66 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   ano                              149258 non-null  int64         
 1   id_cnes                          149253 non-null  object        
 2   target                           149258 non-null  int64         
 3   obito                            149258 non-null  int64         
 4   flag_caso_encerrado              149258 non-null  int64         
 5   qt                               149258 non-null  int64         
 6   id_unidade_cnes                  147338 non-null  object        
 7   nu_cnpj_mantenedora              122937 non-null  object        
 8   id_pfpj                          147338 non-null  object        
 9   nivel_dep                        147338 non-null  object        
 10  no_razao_social                  147338 non-

In [16]:
output = project_root / "data/4_gold/dim_cnes_w_notifications/dim_cnes_w_notifications.parquet"
save_parquet_in_chunks(sinan_cnes,output,100,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/149258 [00:00<?, ?linhas/s]

✔️ dim_cnes_w_notifications.parquet_part_1.parquet salvo com 14.26 MB (149258 linhas)
✅ Todos os arquivos salvos com sucesso.
